# Imports

In [1]:
import torch
import cgd_utils
import numpy as np

# Game Setup

In [5]:
# Game constants
num_evaders = 2
num_exits = 7
points_per_inspector = [3, 3, 1]
num_inspectors = len(points_per_inspector)

# Computing helper values for calculating expected payoffs
bounds = [0] + list(np.cumsum(points_per_inspector))
inspector_ranges = [(bounds[i], bounds[i+1]) for i in range(num_inspectors)]

# Check that each point has exactly one inspector.
assert(sum(points_per_inspector) == num_exits)

# Calculate game payoffs
def calculate_expected_payoffs(evader_player_list, inspector_player_list):
    '''Given lists of evader probabilities and inspector probabilities, compute expected payoffs'''
    
    # Normalize each evader's probabilities to [0, 1] using softmax.
    normalized_evader_list = (
        [torch.nn.Softmax(dim=0)(evader_tensor)
         for evader_tensor in evader_player_list])
    
    # Normalize each inspector's probabilities to [0,1] using softmax.
    normalized_inspector_list = (
        [torch.nn.Softmax(dim=0)(inspect_tensor)
         for inspect_tensor in inspector_player_list])
     
    # Evader probability of gaining 1 is probability that inspector does not choose this point.
    inspector_probabilities = 1 - torch.cat(normalized_inspector_list)
    print(inspector_probabilities)
    # Get evader expected payoffs in-order of evader, which is probability that
    # evader choses a point, that the inspector does not (i.e. complement)
    evader_payoffs = [-torch.dot(evader_tensor, inspector_probabilities) 
                      for evader_tensor in normalized_evader_list]
    
    # Define list of inspector payoffs
    inspector_payoffs = [torch.tensor(0.) 
                         for _ in range(num_inspectors)]
    
    # For inspector, payoff is probability that evader and inspector both chose the same exit.
    for evader_tensor in normalized_evader_list:
        for i, ((start, stop), inspector_tensor) in (
                enumerate(zip(inspector_ranges, normalized_inspector_list))):
            
            inspector_payoffs[i] += -torch.dot(inspector_tensor, evader_tensor[start: stop])
            
    return evader_payoffs, inspector_payoffs
            

# Simulating Invader Evader with Random Starting Weights

In [6]:
num_iterations = 2000
learning_rates = [0.01] * (num_evaders + num_inspectors)

# Define initial probability-ish tensors for evaders and invaders
evader_player_list = [torch.nn.Softmax(dim=0)(torch.tensor(np.random.uniform(size=(num_exits)), 
                                   requires_grad=True))
                      for _ in range(num_evaders)]

inspector_player_list = [torch.nn.Softmax(dim=0)(torch.tensor(np.random.uniform(size=(num_points)), 
                                      requires_grad=True))
                          for num_points in points_per_inspector]

print("Initial Evader Probabilities")
for tensor in evader_player_list:
    print("   " + str(tensor))

print()
    
print("Initial Inspector Probabilities")
for tensor in inspector_player_list:
    print("    " + str(tensor))

# Simulate multiple rounds of game
for i in range(num_iterations):
    if (i % 100 == 0):
        print(i)
        
    evader_payoffs, inspector_payoffs = calculate_expected_payoffs(evader_player_list, inspector_player_list)
    
    updates, _ = cgd_utils.metamatrix_conjugate_gradient(
        evader_payoffs + inspector_payoffs, 
        evader_player_list + inspector_player_list, 
        lr_list=learning_rates)
    
    for player, update in zip(evader_player_list + inspector_player_list, updates):
        player.data.add_(update)
    
    # Normalize probabilities for next iteration with softmax.
    evader_player_list = (
        [torch.nn.Softmax(dim=0)(evader_tensor)
         for evader_tensor in evader_player_list])
    inspector_player_list = (
        [torch.nn.Softmax(dim=0)(inspect_tensor)
         for inspect_tensor in inspector_player_list])
    
    evader_player_list = [tensor.clone().detach().requires_grad_(True) for tensor in evader_player_list]
    inspector_player_list = [tensor.clone().detach().requires_grad_(True) for tensor in inspector_player_list] 

# Look at final normalized probabilities using softmax.
normalized_evader_list = (
        [torch.nn.Softmax(dim=0)(evader_tensor) 
         for evader_tensor in evader_player_list])
normalized_inspector_list = (
        [torch.nn.Softmax(dim=0)(inspect_tensor)
         for inspect_tensor in inspector_player_list])

print("Final Evader Probabilities")
for tensor in normalized_evader_list:
    print("   " + str(tensor))

print()
    
print("Final Inspector Probabilities")
for tensor in normalized_inspector_list:
    print("    " + str(tensor))

Initial Evader Probabilities
   tensor([0.2084, 0.1655, 0.0827, 0.1424, 0.1976, 0.1167, 0.0866],
       dtype=torch.float64, grad_fn=<SoftmaxBackward>)
   tensor([0.1161, 0.1414, 0.1870, 0.1311, 0.1479, 0.1318, 0.1447],
       dtype=torch.float64, grad_fn=<SoftmaxBackward>)

Initial Inspector Probabilities
    tensor([0.5374, 0.2356, 0.2271], dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
    tensor([0.4300, 0.3559, 0.2141], dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
    tensor([1.], dtype=torch.float64, grad_fn=<SoftmaxBackward>)
0
tensor([0.5956, 0.7009, 0.7035, 0.6343, 0.6604, 0.7053, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6426, 0.6783, 0.6791, 0.6558, 0.6647, 0.6794, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6586, 0.6706, 0.6708, 0.6631, 0.6660, 0.6709, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6640, 0.6680, 0.6681, 0.6655, 0.6665, 0.6681, 0.0000],
       dtype=torch.flo

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

300
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.66

...

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.0000],
       dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor([0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 

# Simulation with Initial Nash Equilbrium Weights

In [4]:
num_iterations = 1000
learning_rates = [1] * (num_evaders + num_inspectors)

# Define initial probability-ish tensors for evaders and invaders
evader_player_list = [torch.tensor([1/6] * num_exits, 
                                   requires_grad=True) 
                      for _ in range(num_evaders)]

inspector_player_list = [torch.tensor([1/2] * num_points, 
                                      requires_grad=True) 
                          for num_points in points_per_inspector]

# Simulate multiple rounds of game
for i in range(num_iterations):
#     if (i % 10 == 0):
#         print(i)
#         print(evader_player_list)
    evader_payoffs, inspector_payoffs = calculate_expected_payoffs(evader_player_list, inspector_player_list)
    
    print(evader_payoffs, inspector_payoffs)
    
    updates, _ = cgd_utils.metamatrix_conjugate_gradient(
        evader_payoffs + inspector_payoffs, 
        evader_player_list + inspector_player_list, 
        lr_list=learning_rates)
    
    
    for player, update in zip(evader_player_list + inspector_player_list, updates):
        player.data.add_(update)

# Look at final normalized probabilities
normalized_evader_list = (
        [evader_tensor / torch.norm(evader_tensor, 1) 
         for evader_tensor in evader_player_list])
normalized_inspector_list = (
        [inspect_tensor / torch.norm(inspect_tensor, 1) 
         for inspect_tensor in inspector_player_list])

print("Evader Probabilities")
for tensor in normalized_evader_list:
    print("   " + str(tensor))

print()
    
print("Inspector Probabilities")
for tensor in normalized_inspector_list:
    print("    " + str(tensor))

[tensor(-0.5714, grad_fn=<NegBackward>), tensor(-0.5714, grad_fn=<NegBackward>)] [tensor(-0.2857, grad_fn=<AddBackward0>), tensor(-0.2857, grad_fn=<AddBackward0>), tensor(-0.2857, grad_fn=<AddBackward0>)]
[tensor(-0.5789, grad_fn=<NegBackward>), tensor(-0.5789, grad_fn=<NegBackward>)] [tensor(-0.2895, grad_fn=<AddBackward0>), tensor(-0.2895, grad_fn=<AddBackward0>), tensor(-0.2632, grad_fn=<AddBackward0>)]
[tensor(-0.5855, grad_fn=<NegBackward>), tensor(-0.5855, grad_fn=<NegBackward>)] [tensor(-0.2927, grad_fn=<AddBackward0>), tensor(-0.2927, grad_fn=<AddBackward0>), tensor(-0.2435, grad_fn=<AddBackward0>)]
[tensor(-0.5912, grad_fn=<NegBackward>), tensor(-0.5912, grad_fn=<NegBackward>)] [tensor(-0.2956, grad_fn=<AddBackward0>), tensor(-0.2956, grad_fn=<AddBackward0>), tensor(-0.2263, grad_fn=<AddBackward0>)]
[tensor(-0.5963, grad_fn=<NegBackward>), tensor(-0.5963, grad_fn=<NegBackward>)] [tensor(-0.2982, grad_fn=<AddBackward0>), tensor(-0.2982, grad_fn=<AddBackward0>), tensor(-0.2111, 

[tensor(-0.6491, grad_fn=<NegBackward>), tensor(-0.6491, grad_fn=<NegBackward>)] [tensor(-0.3246, grad_fn=<AddBackward0>), tensor(-0.3246, grad_fn=<AddBackward0>), tensor(-0.0526, grad_fn=<AddBackward0>)]
[tensor(-0.6495, grad_fn=<NegBackward>), tensor(-0.6495, grad_fn=<NegBackward>)] [tensor(-0.3247, grad_fn=<AddBackward0>), tensor(-0.3247, grad_fn=<AddBackward0>), tensor(-0.0515, grad_fn=<AddBackward0>)]
[tensor(-0.6498, grad_fn=<NegBackward>), tensor(-0.6498, grad_fn=<NegBackward>)] [tensor(-0.3249, grad_fn=<AddBackward0>), tensor(-0.3249, grad_fn=<AddBackward0>), tensor(-0.0506, grad_fn=<AddBackward0>)]
[tensor(-0.6501, grad_fn=<NegBackward>), tensor(-0.6501, grad_fn=<NegBackward>)] [tensor(-0.3251, grad_fn=<AddBackward0>), tensor(-0.3251, grad_fn=<AddBackward0>), tensor(-0.0496, grad_fn=<AddBackward0>)]
[tensor(-0.6504, grad_fn=<NegBackward>), tensor(-0.6504, grad_fn=<NegBackward>)] [tensor(-0.3252, grad_fn=<AddBackward0>), tensor(-0.3252, grad_fn=<AddBackward0>), tensor(-0.0487, 

[tensor(-0.6570, grad_fn=<NegBackward>), tensor(-0.6570, grad_fn=<NegBackward>)] [tensor(-0.3285, grad_fn=<AddBackward0>), tensor(-0.3285, grad_fn=<AddBackward0>), tensor(-0.0290, grad_fn=<AddBackward0>)]
[tensor(-0.6571, grad_fn=<NegBackward>), tensor(-0.6571, grad_fn=<NegBackward>)] [tensor(-0.3286, grad_fn=<AddBackward0>), tensor(-0.3286, grad_fn=<AddBackward0>), tensor(-0.0287, grad_fn=<AddBackward0>)]
[tensor(-0.6572, grad_fn=<NegBackward>), tensor(-0.6572, grad_fn=<NegBackward>)] [tensor(-0.3286, grad_fn=<AddBackward0>), tensor(-0.3286, grad_fn=<AddBackward0>), tensor(-0.0284, grad_fn=<AddBackward0>)]
[tensor(-0.6573, grad_fn=<NegBackward>), tensor(-0.6573, grad_fn=<NegBackward>)] [tensor(-0.3287, grad_fn=<AddBackward0>), tensor(-0.3287, grad_fn=<AddBackward0>), tensor(-0.0281, grad_fn=<AddBackward0>)]
[tensor(-0.6574, grad_fn=<NegBackward>), tensor(-0.6574, grad_fn=<NegBackward>)] [tensor(-0.3287, grad_fn=<AddBackward0>), tensor(-0.3287, grad_fn=<AddBackward0>), tensor(-0.0278, 

[tensor(-0.6603, grad_fn=<NegBackward>), tensor(-0.6603, grad_fn=<NegBackward>)] [tensor(-0.3301, grad_fn=<AddBackward0>), tensor(-0.3301, grad_fn=<AddBackward0>), tensor(-0.0192, grad_fn=<AddBackward0>)]
[tensor(-0.6603, grad_fn=<NegBackward>), tensor(-0.6603, grad_fn=<NegBackward>)] [tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.0191, grad_fn=<AddBackward0>)]
[tensor(-0.6604, grad_fn=<NegBackward>), tensor(-0.6604, grad_fn=<NegBackward>)] [tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.0189, grad_fn=<AddBackward0>)]
[tensor(-0.6604, grad_fn=<NegBackward>), tensor(-0.6604, grad_fn=<NegBackward>)] [tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.0188, grad_fn=<AddBackward0>)]
[tensor(-0.6604, grad_fn=<NegBackward>), tensor(-0.6604, grad_fn=<NegBackward>)] [tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.3302, grad_fn=<AddBackward0>), tensor(-0.0187, 

[tensor(-0.6617, grad_fn=<NegBackward>), tensor(-0.6617, grad_fn=<NegBackward>)] [tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.0148, grad_fn=<AddBackward0>)]
[tensor(-0.6617, grad_fn=<NegBackward>), tensor(-0.6617, grad_fn=<NegBackward>)] [tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.0148, grad_fn=<AddBackward0>)]
[tensor(-0.6618, grad_fn=<NegBackward>), tensor(-0.6618, grad_fn=<NegBackward>)] [tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.0147, grad_fn=<AddBackward0>)]
[tensor(-0.6618, grad_fn=<NegBackward>), tensor(-0.6618, grad_fn=<NegBackward>)] [tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.0146, grad_fn=<AddBackward0>)]
[tensor(-0.6618, grad_fn=<NegBackward>), tensor(-0.6618, grad_fn=<NegBackward>)] [tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.3309, grad_fn=<AddBackward0>), tensor(-0.0145, 

...

[tensor(-0.6656, grad_fn=<NegBackward>), tensor(-0.6656, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0031, grad_fn=<AddBackward0>)]
[tensor(-0.6656, grad_fn=<NegBackward>), tensor(-0.6656, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0031, grad_fn=<AddBackward0>)]
[tensor(-0.6656, grad_fn=<NegBackward>), tensor(-0.6656, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0031, grad_fn=<AddBackward0>)]
[tensor(-0.6656, grad_fn=<NegBackward>), tensor(-0.6656, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0031, grad_fn=<AddBackward0>)]
[tensor(-0.6656, grad_fn=<NegBackward>), tensor(-0.6656, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0031, 

[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0030, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0030, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0030, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0030, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.3328, grad_fn=<AddBackward0>), tensor(-0.0030, 

[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0028, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0028, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0028, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0028, grad_fn=<AddBackward0>)]
[tensor(-0.6657, grad_fn=<NegBackward>), tensor(-0.6657, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0028, 

[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0027, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0027, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0027, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0027, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0027, 

[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0026, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0026, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0026, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0026, grad_fn=<AddBackward0>)]
[tensor(-0.6658, grad_fn=<NegBackward>), tensor(-0.6658, grad_fn=<NegBackward>)] [tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.3329, grad_fn=<AddBackward0>), tensor(-0.0026, 